### Installation

In [2]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

### Unsloth

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.12.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(example):
    text = tokenizer.apply_chat_template(
        example["conversations"],
        tokenize=False,
        add_generation_prompt=False
    )
    return [text]  # ← Flat list of strings!


In [ ]:
from google.colab import files
uplodaed = files.upload()

In [6]:
# === YOUR KAGGLE DATASET ===
import pandas as pd
from datasets import Dataset

# Load your CSV (adjust path/columns)
df = pd.read_csv("train.csv")  # e.g. columns: "ancient", "english"
print(df.head())  # Verify columns

def make_translation_pair(row):
    return {
        "conversations": [  # Keep
            {"role": "system", "content": "You are an expert translator of Akkadian language to English. Translate accurately."},
            {"role": "user", "content": f"Translate this Akkadian sentence to English:\n\n{row['transliteration']}"},
            {"role": "assistant", "content": row['translation']},
        ]
    }


dataset = Dataset.from_list([make_translation_pair(row) for _, row in df.iterrows()])

# Train/test split
dataset = dataset.train_test_split(test_size=0.1, seed=3407)
print(f"Train: {len(dataset['train'])}, Eval: {len(dataset['test'])}")

                                oare_id  \
0  004a7dbd-57ce-46f8-9691-409be61c676e   
1  0064939c-59b9-4448-a63d-34612af0a1b5   
2  0073f2c0-524c-4bbf-915a-8c1772a4fb98   
3  009fb838-8038-42bc-ad34-5f795b3840ee   
4  00aa1c55-c80c-4346-a159-73ad43ab0ff7   

                                     transliteration  \
0  KIŠIB ma-nu-ba-lúm-a-šur DUMU ṣí-lá-(d)IM KIŠI...   
1               1 TÚG ša qá-tim i-tur₄-DINGIR il₅-qé   
2  TÚG u-la i-dí-na-ku-um i-tù-ra-ma 9 GÍN KÙ.BABBAR   
3  KIŠIB šu-(d)EN.LÍL DUMU šu-ku-bi-im KIŠIB ṣí-l...   
4  um-ma šu-ku-tum-ma a-na IŠTAR-lá-ma-sí ù ni-ta...   

                                         translation  
0  Seal of Mannum-balum-Aššur son of Ṣilli-Adad, ...  
1  Itūr-ilī has received one textile of ordinary ...  
2  ... he did not give you a textile. He returned...  
3  Seal of Šu-Illil son of Šu-Kūbum, seal of Ṣilū...  
4  From Šukkutum to Ištar-lamassī and Nitahšušar:...  
Train: 1404, Eval: 157


In [7]:
def add_text(example):
    example["text"] = tokenizer.apply_chat_template(
        example["conversations"],
        tokenize=False,
        add_generation_prompt=False
    )
    return example

dataset = dataset.map(add_text, batched=False)


Map:   0%|          | 0/1404 [00:00<?, ? examples/s]

Map:   0%|          | 0/157 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [8]:
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorForSeq2Seq
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    eval_dataset=dataset["test"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/1404 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/157 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [9]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=6):   0%|          | 0/1404 [00:00<?, ? examples/s]

Map (num_proc=6):   0%|          | 0/157 [00:00<?, ? examples/s]

We can see the System and Instruction prompts are successfully masked!

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,404 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.930100
2,2.826300
3,2.949000
4,1.944000
5,2.696400
6,2.136000
7,2.240200
8,1.724600
9,2.018900
10,2.191100


In [11]:
# Extract test transliterations & references
test_transliterations = [ex["conversations"][1]["content"] for ex in dataset["test"]]
test_references = [ex["conversations"][2]["content"] for ex in dataset["test"]]

print(f"Test set: {len(test_transliterations)} samples")
print("Sample:", test_transliterations[0][:100] + "...")


Test set: 157 samples
Sample: Translate this Akkadian sentence to English:

a-na a-lá-ḫi-im qí-bi-ma um-ma SIG₅-pí-a-šur-ma a-dí š...


In [12]:
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")
FastLanguageModel.for_inference(model)

def translate_akkadian_batched(transliterations, batch_size=4):  # Smaller batch for T4
    predictions = []
    for i in range(0, len(transliterations), batch_size):
        batch = transliterations[i:i+batch_size]
        messages = [{"role": "user", "content": f"Translate this Akkadian sentence to English:\n\n{t}"} for t in batch]

        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            padding=True,
            return_tensors="pt",
        ).to("cuda")

        # Fixed: use inputs directly (it's already input_ids tensor)
        outputs = model.generate(
            inputs,
            max_new_tokens=150,
            temperature=0.1,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

        batch_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        batch_preds = [p.split("assistant")[-1].split("<|eot_id|>")[0].strip() for p in batch_preds]
        predictions.extend(batch_preds)
        print(f"Batch {i//batch_size+1}/{len(transliterations)//batch_size+1}")
    return predictions

test_predictions = translate_akkadian_batched(test_transliterations)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Batch 1/40
Batch 2/40
Batch 3/40
Batch 4/40
Batch 5/40
Batch 6/40
Batch 7/40
Batch 8/40
Batch 9/40
Batch 10/40
Batch 11/40
Batch 12/40
Batch 13/40
Batch 14/40
Batch 15/40
Batch 16/40
Batch 17/40
Batch 18/40
Batch 19/40
Batch 20/40
Batch 21/40
Batch 22/40
Batch 23/40
Batch 24/40
Batch 25/40
Batch 26/40
Batch 27/40
Batch 28/40
Batch 29/40
Batch 30/40
Batch 31/40
Batch 32/40
Batch 33/40
Batch 34/40
Batch 35/40
Batch 36/40
Batch 37/40
Batch 38/40
Batch 39/40
Batch 40/40


In [56]:
test_references = [ex["conversations"][2]["content"] for ex in dataset["test"]]


In [13]:
!pip install sacrebleu sacrebleu[chrF] -q
# OR standalone chrF
!pip install chrF -q


ERROR: Could not find a version that satisfies the requirement chrF (from versions: none)
ERROR: No matching distribution found for chrF


In [14]:
import sacrebleu
import numpy as np

bleu = sacrebleu.corpus_bleu(test_predictions, [test_references])

# chrF++ via sacrebleu (6-char + 2-word n-grams = official)
chrf_pp = sacrebleu.corpus_chrf(
    test_predictions,
    [test_references],
    char_order=6,
    word_order=2
)

geo_mean = np.sqrt(bleu.score * chrf_pp.score)
print(f"BLEU:     {bleu.score:.2f}")
print(f"chrF++:   {chrf_pp.score:.2f}")
print(f"Geo Mean: {geo_mean:.2f}  ← **KAGGLE SCORE**")

BLEU:     2.24
chrF++:   14.54
Geo Mean: 5.71  ← **KAGGLE SCORE**


In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving